## Areal Independence verifcation

This notebook is designed to help numerically explore which parameters are area-independent and which are not. Before getting started, please ensure that you've installed all the dependencies specified in `requirements.txt`, located at the root of this directory. If you have access to a terminal, simply run `pip install -r requirements.txt`

In [1]:
import os
import sys
sys.path.append('../')
from functools import partial

import numpy as np
import pandas as pd
import pathos.multiprocessing as mp
from tqdm import tqdm

from dr.load import load_table
from dr.extraction import parallelized_extraction, extract_parameters
from dr.utils import fit_and_subtract, get_fourier_spectrum

In [2]:
# Zoom function
def zoom(arr, quadrant=1):
    """Slices a 2D array to return a particular quadrant of it.

    Args:
        arr (np.array): The MxN 2D array.
        quadrant (int): The quadrant of the image to extract. Quadrant          1 maps to upper left, 2 to upper right, 3 to lower left, and 
        4 to lower right. Anything else extracts the middle of `arr`.
        
    Returns:
        np.array: A `M // 2` x `N // 2` quadrant of `arr`
    """
    M, N = arr.shape
    fitted = arr
    if quadrant == 1:
        return fitted[:M // 2, :N // 2]
    elif quadrant == 2:
        return fitted[M // 2:, :N // 2]
    elif quadrant == 3:
        return fitted[:M // 2, N // 2:]
    elif quadrant == 4:
        return fitted[M // 2:, N // 2:]
    else:
        # Slice out the middle of arr
        return fitted[M - M // 2:M + M // 2, N - N // 2:N + N // 2]

# This function is similar to parallelized_extraction but it only extracts S_rw
def calculate_srw(fnames, dx, M, f=None):
    from dr.parameters.spatial import S_rw
    def extract(fname):
        channel = np.loadtxt(fname)
        if f is not None:
            channel = f(channel)
        spectrum, _, _ = get_fourier_spectrum(fit_and_subtract(channel))
        return S_rw(spectrum, dx, M)

    with mp.Pool(processes=PROCESSES) as pool:
        result = list(tqdm(pool.imap(extract, fnames), total=len(fnames)))
    return pd.DataFrame(result)


In [3]:
# Number of processes to use for parameter calculation
PROCESSES = 6 # (os.cpu_count() - 1) or (os.cpu_count() / 2) are good choices
# Pixel separation distance, irrelevant for simulated data
DELTA = 5 / 256
# Number of simulated surfaces to calculate parameters for
NUM = 20

# Zoom function to apply
FUNC = partial(zoom, quadrant=5)

# Filenames for surfaces
fnames = [f'../data/simulated/test{i}.asc' for i in range(NUM)]

In [4]:
df_1 = calculate_srw(fnames, DELTA, 256)
df_2 = calculate_srw(fnames, DELTA, 256, f=FUNC)

100%|██████████| 20/20 [00:04<00:00,  4.10it/s]


In [5]:
(df_1 / df_2).mean()

0    1.055879
dtype: float64

In [6]:
# Calculate parameters on full surface; M corresponds to x-dimension of the surface
df_full = parallelized_extraction(fnames, dx=DELTA, dy=DELTA, M=256, processes=PROCESSES)

100%|██████████| 20/20 [00:46<00:00,  2.32s/it]


In [7]:
# Calculate parameters on a subset of the surface (in this case, the upper-left quadrant)
df_quarter = parallelized_extraction(fnames, dx=DELTA, dy=DELTA, M=256, processes=PROCESSES, f=FUNC)

100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


In [8]:
df_full

,S_a,S_q,S_sk,S_ku,S_z,S_10z,S_v,S_p,S_mean,S_sc,...,S_td,S_tdi,S_rw,S_rwi,S_hw,S_fd,S_cl20,S_cl37,S_tr20,S_tr37
0,373.360652,476.341942,-0.349412,2.651794,2601.824932,6860.130978,-1417.546367,1184.278565,1043.217474,69229.976195,...,90.000000,0.542723,4.980469,0.016683,4.980469,2.307348,0.013811,0.013811,102.005852,124.095709
1,90.531954,121.401499,1.057788,4.556371,901.447453,3134.213334,-326.699682,574.747771,128.852964,86937.004648,...,90.000000,0.255791,4.980469,0.037124,1.660156,2.466302,0.013811,0.013811,128.341425,141.925765
2,41.518735,57.989376,1.154854,5.242922,376.304745,1400.493917,-109.261686,267.043058,56.649666,15524.156202,...,0.000000,0.359932,4.980469,0.023672,2.490234,2.340202,0.013811,0.013811,54.119605,151.805470
3,138.465484,166.263054,0.419556,2.681135,994.671639,3237.115668,-406.735370,587.936269,231.768163,51850.195943,...,0.703125,0.400538,4.980469,0.015768,4.980469,2.387842,0.013811,0.013811,97.556347,100.663763
4,373.176186,448.639699,0.170542,2.292477,2239.104941,7220.182290,-960.224115,1278.880826,572.014440,88817.145113,...,135.000000,0.551666,4.980469,0.017144,4.980469,2.349168,0.013811,0.013811,108.657794,100.790158
5,313.929724,407.530718,0.545095,3.213814,2722.999081,5649.756704,-1078.532670,1644.466411,499.162089,13268.002368,...,135.000000,0.545213,4.980469,0.015685,4.980469,2.285230,0.013811,0.013811,103.099081,108.855906
6,403.488484,515.049798,0.880894,3.406838,2732.039591,9300.640630,-1027.063517,1704.976074,469.181926,10111.981226,...,90.000000,0.360826,4.980469,0.032058,1.660156,2.207931,0.013811,0.013811,78.364308,54.626833
7,295.084943,380.522780,1.030362,4.000379,2234.223494,7248.810324,-606.112250,1628.111245,366.211705,48541.538573,...,90.000000,0.266807,4.980469,0.019215,4.980469,2.295042,0.013811,0.013811,84.623465,63.707600
8,206.786244,250.534716,0.542447,2.500261,1221.222366,4106.513383,-485.596341,735.626025,272.208947,31428.760122,...,120.234375,0.495417,2.490234,0.023050,2.490234,2.304474,0.013811,0.013811,85.093169,45.846183
9,559.638965,668.025250,-0.268475,2.191652,2835.193525,7975.752611,-1542.432398,1292.761127,1356.301753,66277.870624,...,0.000000,0.316270,4.980469,0.022941,2.490234,2.284199,0.013811,0.013811,76.666602,48.777309


In [9]:
df_quarter

,S_a,S_q,S_sk,S_ku,S_z,S_10z,S_v,S_p,S_mean,S_sc,...,S_td,S_tdi,S_rw,S_rwi,S_hw,S_fd,S_cl20,S_cl37,S_tr20,S_tr37
0,172.942432,212.575347,0.180010,2.543431,1180.157772,3307.565225,-624.344081,555.813690,1753.048813,72139.196093,...,0.000000,0.327312,4.980469,0.027739,1.657998,2.351922,0.013811,0.013811,128.000000,84.335928
1,24.801619,31.755965,1.251173,5.124035,271.221187,1006.946664,-46.500017,224.721170,39.731864,64263.908635,...,130.078125,0.703690,4.980469,0.122640,0.135086,2.738722,0.013811,0.013811,1.414214,128.000000
2,29.437221,38.457493,-0.634625,4.670384,297.018865,720.951520,-176.715643,120.303222,78.649038,18024.942194,...,135.000000,0.431842,3.329050,0.051869,0.996897,2.335921,0.013811,0.013811,76.672919,128.000000
3,71.246735,90.284165,-0.295444,3.372815,774.426323,1756.941032,-473.796244,300.630078,487.801412,64070.462278,...,0.000000,0.262843,4.980469,0.055556,0.623801,2.426442,0.013811,0.013811,72.939686,117.637428
4,222.561323,277.680894,0.633833,3.060133,1560.803492,4930.514451,-668.796577,892.006915,880.033581,97806.099671,...,135.000000,0.297377,4.980469,0.062113,0.831186,2.331131,0.013811,0.013811,128.000000,101.849671
5,155.433819,208.036910,0.380158,3.862944,1289.526836,2782.318915,-599.213731,690.313106,819.372468,22034.544627,...,178.593750,0.550041,3.329050,0.044560,1.245136,2.164915,0.013811,0.013811,128.000000,75.932253
6,578.829413,686.971740,0.099511,2.193934,2923.339617,6332.915401,-1441.685719,1481.653898,939.359173,19233.385690,...,135.000000,0.483695,3.329050,0.043759,1.245136,2.191204,0.013811,0.013811,47.209076,111.666325
7,165.549289,219.061304,1.318055,6.052523,1435.729775,4808.925759,-402.532271,1033.197503,298.163577,44920.626298,...,88.593750,0.468940,4.980469,0.031540,1.245136,2.285387,0.013811,0.013811,103.462889,128.000000
8,223.629030,270.128130,0.726211,2.768909,1287.787537,4422.770338,-462.455538,825.332000,372.839640,36257.147804,...,90.000000,0.376577,3.329050,0.035194,1.245136,2.215125,0.013811,0.013811,46.425358,128.000000
9,346.403264,431.489977,-0.757374,3.264600,2394.767621,5959.100139,-1420.197409,974.570212,2257.183214,70808.855112,...,135.000000,0.521346,4.980469,0.032502,1.657998,2.222869,0.013811,0.013811,38.581992,74.203089


In [10]:
(df_full / df_quarter).mean()

S_a           1.403438
S_q           1.427638
S_sk          0.671974
S_ku          1.070836
S_z           1.418493
S_10z         1.585780
S_v           1.412268
S_p           1.661491
S_mean        0.809080
S_sc          0.880314
S_2a          4.000000
S_3a          4.250115
S_dr          1.062529
S_dq          1.000000
S_dq6         0.999958
S_bi          0.889627
S_ci          1.162375
S_vi          1.254110
S_pk          3.316286
S_vk          2.126481
S_k           1.406882
S_dc0-5       2.061842
S_dc5-10      2.100988
S_dc10-50     1.626604
S_dc50-95     1.430724
S_dc50-100    1.448045
S_ds          1.770798
S_td               inf
S_tdi         0.943242
S_rw          1.055879
S_rwi         0.631195
S_hw          3.241504
S_fd          1.006227
S_cl20        1.000000
S_cl37        1.000000
S_tr20        5.935043
S_tr37        1.192719
dtype: float64

In [11]:
# As these are Pandas DataFrames, you may index them by column, like so:
(df_full / df_quarter)['S_rw']

0     1.000000
1     1.000000
2     1.496063
3     1.000000
4     1.000000
5     1.496063
6     1.496063
7     1.000000
8     0.748031
9     1.000000
10    0.398425
11    1.496063
12    1.496063
13    0.748031
14    0.498688
15    1.000000
16    1.000000
17    1.000000
18    0.748031
19    1.496063
Name: S_rw, dtype: float64

### Warning about simulated surface results

The results offered by testing this on simulated surfaces may be somewhat noisy, as most of the differences arise from variance of the data, as opposed to areal dependence. For instance, the ratios for S_a generally do not fall around 1, but we know analytically that it is area-independent (in the sense that it is normalized w.r.t. area). Using real surfaces seems to offer cleaner results.

In [12]:
# Load data for real surfaces
channel_c_srk = load_table('data/cells_c-srk.txt')['Adhesion'].values.reshape(512, 512) # 512 x  512
channel_h7_29 = load_table('data/cells_h7-29.txt')['Adhesion'].values.reshape(256, 256) # 256 x 256

channel_c_srk_zoomed = zoom(channel_c_srk, quadrant=1) # 256 x 256
channel_h7_29_zoomed = zoom(channel_h7_29, quadrant=1) # 128 x 128

FileNotFoundError: [Errno 2] No such file or directory: 'data/cells_c-srk.txt'

In [13]:
# May take some time to extract all the parameters in this manner
c_srk_results = extract_parameters(channel_c_srk, dx=DELTA, dy=DELTA, M=512)
h7_29_results = extract_parameters(channel_h7_29, dx=DELTA, dy=DELTA, M=256)

c_srk_zoomed_results = extract_parameters(channel_c_srk_zoomed, dx=DELTA, dy=DELTA, M=256)
h7_29_zoomed_results = extract_parameters(channel_h7_29_zoomed, dx=DELTA, dy=DELTA, M=128)

NameError: name 'channel_c_srk' is not defined

Of particular interest are the parameters `S_rw`, `S_rwi`, and `S_rwi`. Parameters in this list can be verified to be area-independent either numerically (i.e. the ratio is 1) or analytically. Note that not all parameters match the original definition; for instance, `S_clxx` is normalized to help make it area independent.

In [14]:
c_srk_results / c_srk_zoomed_results

NameError: name 'c_srk_results' is not defined

In [15]:
h7_29_results / h7_29_zoomed_results

NameError: name 'h7_29_results' is not defined

### Additional discussion

While it's clear that `S_rw` consistently varies between the full image and its zoom by a factor of 2, the range of possible values for `S_rw` depends on the dimensions of the image because it is measuring a radius. So, on a 512x512 image `S_rw` can range from `0.01953125` (i.e. the pixel separation distance) to `9.98046875`. However, on a 256x256 image `S_rw` can only range from `0.01953125` to `4.98046875`. 

We can ensure that the ranges are the same by using the same value for the parameter `M` in each calculation, but doing this will result in a loss of information for some images. For instance, if we let `M = 256`, then `S_rw` can only be calculated on effectively half of a 512x512 image.